<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%EB%8C%80%ED%99%94%ED%98%95_%EC%8A%B5%EA%B3%B5%EA%B8%B0%EC%84%A0%EB%8F%84_(%ED%8C%8C%EC%9D%BC_%EC%84%A0%ED%83%9D%2C_%EB%8F%99%EC%A0%81_%EC%BB%AC%EB%9F%BC_%EB%A7%A4%ED%95%91%2C_%EB%A6%AC%EC%83%98%ED%94%8C%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import io # 문자열로부터 CSV를 읽기 위한 모듈
import os # 실제 파일 시스템 상호작용을 위한 모듈 (현재는 주석 처리)
import glob # 실제 파일 시스템 상호작용을 위한 모듈 (현재는 주석 처리)

# 1. 기본 상수 정의
C_PA = 1.006  # 건공기의 정압비열 (kJ/kg°C)
C_PW = 1.86   # 수증기의 정압비열 (kJ/kg°C)
H_FG0 = 2501  # 0°C에서의 물의 증발잠열 (kJ/kg)
P_ATM = 101.325 # 표준 대기압 (kPa)

# 2. 습공기 상태량 계산 함수 - 변경 없음
def get_saturation_vapor_pressure_kPa(temp_c):
    if temp_c < -60: return 0.0
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

def get_actual_vapor_pressure_kPa(temp_c, rh_percent):
    p_ws = get_saturation_vapor_pressure_kPa(temp_c)
    return (rh_percent / 100.0) * p_ws

def get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    p_w = get_actual_vapor_pressure_kPa(temp_c, rh_percent)
    denominator = p_atm_kPa - p_w
    if denominator <= 1e-9:
        if abs(p_w - get_saturation_vapor_pressure_kPa(temp_c)) < 1e-6 and rh_percent > 99.99:
             p_w_adjusted = get_saturation_vapor_pressure_kPa(temp_c) * 0.99999
             denominator_adjusted = p_atm_kPa - p_w_adjusted
             if denominator_adjusted <= 1e-9: return float('inf')
             return 0.621945 * p_w_adjusted / denominator_adjusted
        else:
            return float('inf')
    return 0.621945 * p_w / denominator

def get_enthalpy_kJ_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    w = get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa)
    if w == float('inf'): return float('inf')
    return C_PA * temp_c + w * (H_FG0 + C_PW * temp_c)

def get_temp_from_enthalpy_humidity_ratio(h_kJ_kg, w_kg_kg):
    denominator = C_PA + w_kg_kg * C_PW
    if abs(denominator) < 1e-9: return float('nan')
    return (h_kJ_kg - w_kg_kg * H_FG0) / denominator

# 3. Plotly용 습공기선도 배경 그리기 함수 - 이전 Plotly 버전과 동일
def add_psychrometric_background_to_fig(fig, temp_range_c, w_range_kg_kg):
    temps_c_plot = np.linspace(temp_range_c[0], temp_range_c[1], 100)
    w_saturated = np.array([get_humidity_ratio_kg_kg(t, 100) for t in temps_c_plot])
    valid_sat_indices = ~np.isinf(w_saturated) & (w_saturated <= w_range_kg_kg[1])
    fig.add_trace(go.Scatter(
        x=temps_c_plot[valid_sat_indices], y=w_saturated[valid_sat_indices],
        mode='lines', name='상대습도 100%', line=dict(color='black', width=1.5),
        hoverinfo='skip', showlegend=True
    ))
    rh_curves_to_plot = [20, 40, 60, 80]
    for rh_val in rh_curves_to_plot:
        w_rh_plot = np.array([get_humidity_ratio_kg_kg(t, rh_val) for t in temps_c_plot])
        valid_rh_indices = ~np.isinf(w_rh_plot) & (w_rh_plot <= w_range_kg_kg[1]) & (w_rh_plot >= w_range_kg_kg[0])
        if np.any(valid_rh_indices):
            fig.add_trace(go.Scatter(
                x=temps_c_plot[valid_rh_indices], y=w_rh_plot[valid_rh_indices],
                mode='lines', name=f'상대습도 {rh_val}%', line=dict(color='blue', width=0.8, dash='dash'),
                hoverinfo='skip', showlegend=True
            ))
    h_min_approx = get_enthalpy_kJ_kg(temp_range_c[0], 20 if temp_range_c[0] > 0 else 80)
    h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 100)
    if h_min_approx == float('inf'): h_min_approx = 0
    if h_max_approx == float('inf'): h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 80, P_ATM)
    if h_max_approx == float('inf'): h_max_approx = 120
    enthalpy_lines_values = np.unique(np.linspace(round(h_min_approx / 10) * 10, round(h_max_approx / 10) * 10, 10).astype(int))
    enthalpy_lines_values = [h for h in enthalpy_lines_values if h != float('inf') and h != float('-inf')]
    for h_val in enthalpy_lines_values:
        if np.isinf(h_val) or np.isnan(h_val): continue
        w_for_h_line = np.linspace(w_range_kg_kg[0], w_range_kg_kg[1], 50)
        temps_for_h_line = np.array([get_temp_from_enthalpy_humidity_ratio(h_val, w) for w in w_for_h_line])
        valid_h_points_indices = ~np.isnan(temps_for_h_line) & \
                                 (temps_for_h_line >= temp_range_c[0]) & \
                                 (temps_for_h_line <= temp_range_c[1]) & \
                                 (w_for_h_line >= w_range_kg_kg[0]) & \
                                 (w_for_h_line <= w_range_kg_kg[1])
        if np.sum(valid_h_points_indices) > 1:
            temps_plot_h = temps_for_h_line[valid_h_points_indices]
            w_plot_h = w_for_h_line[valid_h_points_indices]
            fig.add_trace(go.Scatter(
                x=temps_plot_h, y=w_plot_h, mode='lines',
                name=f'엔탈피={h_val}kJ/kg', line=dict(color='green', width=0.8, dash='dot'),
                hoverinfo='skip', showlegend=False
            ))
            if len(temps_plot_h) > 0:
                text_x, text_y = temps_plot_h[0], w_plot_h[0]
                if temp_range_c[0] <= text_x <= temp_range_c[1] and w_range_kg_kg[0] <= text_y <= w_range_kg_kg[1]:
                    fig.add_annotation(x=text_x, y=text_y, text=f"{h_val}", showarrow=False,
                                       font=dict(color="green", size=9), xanchor="right", yanchor="bottom", textangle=-30)

# --- 파일 및 컬럼 처리 함수 ---
def find_oac_csv_files_simulated(virtual_file_system):
    """'OAC'를 이름에 포함하는 CSV 파일을 시뮬레이션 환경에서 찾습니다."""
    print("시뮬레이션된 파일 검색 중...")
    oac_files = {name: content for name, content in virtual_file_system.items() if "oac" in name.lower() and name.endswith(".csv")}
    if not oac_files:
        print("시뮬레이션된 'OAC' CSV 파일을 찾을 수 없습니다.")
        return {}
    return oac_files

def select_csv_file_simulated(oac_files_dict):
    """사용자에게 시뮬레이션된 파일 선택을 요청합니다."""
    if not oac_files_dict:
        return None, None

    print("\n사용 가능한 'OAC' CSV 파일 (시뮬레이션):")
    file_list = list(oac_files_dict.keys())
    for i, filename in enumerate(file_list):
        print(f"  {i+1}. {filename}")

    while True:
        try:
            choice = int(input(f"파일 번호를 선택하세요 (1-{len(file_list)}): "))
            if 1 <= choice <= len(file_list):
                selected_filename = file_list[choice-1]
                return selected_filename, oac_files_dict[selected_filename]
            else:
                print("잘못된 선택입니다. 목록에 있는 번호를 입력하세요.")
        except ValueError:
            print("잘못된 입력입니다. 숫자를 입력하세요.")

# 동적 컬럼 매핑 설정 (순서 중요: 더 구체적인 것을 먼저)
STAGE_MAPPING_DEFINITIONS = [
    {'id': 'PHC', 'display_name': "1. PH/C (예열)", 'stage_keywords': ["PH/C", "PREHEAT", "예열"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#ff7f0e'},
    {'id': 'PCC', 'display_name': "2. PC/C (예냉)", 'stage_keywords': ["PC/C", "PRECOOL", "예냉"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#2ca02c'},
    {'id': 'CC',  'display_name': "3. C/C (냉각)", 'stage_keywords': ["C/C", "COOLINGCOIL", "냉각코일", "COOLING", "COOL"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#d62728'},
    {'id': 'HC',  'display_name': "4. H/C (가열/재열)", 'stage_keywords': ["H/C", "HEATINGCOIL", "재열코일", "승온", "HEATING", "HEAT", "REHEAT"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#9467bd'}
]

def map_columns_from_csv(df_columns):
    """키워드를 기반으로 CSV 컬럼을 스테이지에 동적으로 매핑합니다."""
    mapped_stages_config = []
    print("\n컬럼 매핑 시도 중...")
    available_columns = list(df_columns) # 원본 컬럼 리스트 복사본 사용

    for stage_def in STAGE_MAPPING_DEFINITIONS:
        found_temp_col = None
        found_rh_col = None

        # 온도 컬럼 찾기
        for col_name in available_columns: # 현재 사용 가능한 컬럼 중에서만 검색
            col_lower = col_name.lower()
            if any(sk.lower() in col_lower for sk in stage_def['stage_keywords']) and \
               stage_def['temp_keyword'].lower() in col_lower:
                if found_temp_col is None:
                    found_temp_col = col_name
                else:
                    print(f"  경고: 스테이지 '{stage_def['display_name']}'에 대해 여러 온도 컬럼 후보가 있습니다. 첫 번째 '{found_temp_col}' 사용 (다른 후보: '{col_name}')")

        # 습도 컬럼 찾기
        for col_name in available_columns: # 현재 사용 가능한 컬럼 중에서만 검색
            # 온도 컬럼으로 이미 선택된 것은 습도 컬럼으로 중복 선택 방지 (컬럼명이 완전히 다른 경우에만 해당)
            if col_name == found_temp_col:
                 continue
            col_lower = col_name.lower()
            if any(sk.lower() in col_lower for sk in stage_def['stage_keywords']) and \
               stage_def['rh_keyword'].lower() in col_lower:
                if found_rh_col is None:
                    found_rh_col = col_name
                else:
                     print(f"  경고: 스테이지 '{stage_def['display_name']}'에 대해 여러 습도 컬럼 후보가 있습니다. 첫 번째 '{found_rh_col}' 사용 (다른 후보: '{col_name}')")

        if found_temp_col and found_rh_col:
            mapped_stages_config.append({
                'name': stage_def['display_name'],
                'temp_col': found_temp_col,
                'rh_col': found_rh_col,
                'color': stage_def['color']
            })
            print(f"  스테이지 매핑 성공: {stage_def['display_name']}")
            print(f"    온도 컬럼: '{found_temp_col}'")
            print(f"    습도 컬럼: '{found_rh_col}'")

            # 사용된 컬럼은 다음 스테이지 매핑에서 제외
            if found_temp_col in available_columns:
                available_columns.remove(found_temp_col)
            if found_rh_col in available_columns: # 온도와 습도 컬럼이 다를 경우
                available_columns.remove(found_rh_col)
        elif found_temp_col or found_rh_col : # 둘 중 하나만 찾아진 경우
             print(f"  스테이지 부분 매핑: {stage_def['display_name']}. 온도 컬럼: {found_temp_col}, 습도 컬럼: {found_rh_col}. 이 스테이지는 차트에 포함되지 않습니다.")
        # else: # 둘 다 못 찾은 경우는 자동으로 넘어감 (출력 불필요)

    if not mapped_stages_config:
        print("경고: CSV 컬럼에서 매핑할 수 있는 스테이지가 없습니다.")
    return mapped_stages_config


# 4. 대화형 차트 생성 함수
def create_interactive_psychrometric_chart_plotly(df, mapped_stages_config, selected_filename):
    if df.empty:
        print("데이터프레임이 비어있어 차트를 생성할 수 없습니다.")
        return
    if not mapped_stages_config:
        print("매핑된 스테이지 설정이 없어 차트를 생성할 수 없습니다.")
        return

    all_temps_for_bounds = []
    all_ws_for_bounds = []
    for index, row in df.iterrows():
        for stage in mapped_stages_config:
            try:
                temp = row[stage['temp_col']]
                rh = row[stage['rh_col']]
                all_temps_for_bounds.append(temp)
                w = get_humidity_ratio_kg_kg(temp, rh)
                if w != float('inf') and w < 0.1:
                    all_ws_for_bounds.append(w)
            except KeyError as e:
                # print(f"경고: 범위 계산 중 스테이지 '{stage['name']}'의 컬럼 '{e}'를 찾을 수 없습니다. 이 데이터 포인트는 건너<0xEB><0><0x8A>니다.")
                continue

    if not all_temps_for_bounds or not all_ws_for_bounds:
        print("차트 범위 계산을 위한 유효한 데이터 포인트가 없습니다. 컬럼 매핑 및 데이터를 확인하세요.")
        min_temp_data, max_temp_data = 0, 40
        min_w_data, max_w_data = 0, 0.025
    else:
        min_temp_data, max_temp_data = min(all_temps_for_bounds), max(all_temps_for_bounds)
        min_w_data, max_w_data = min(all_ws_for_bounds), max(all_ws_for_bounds)

    temp_margin = (max_temp_data - min_temp_data) * 0.1 if (max_temp_data - min_temp_data) > 0 else 5
    w_margin = (max_w_data - min_w_data) * 0.1 if (max_w_data - min_w_data) > 0 else 0.002
    chart_min_temp = min_temp_data - temp_margin
    chart_max_temp = max_temp_data + temp_margin * 1.5
    chart_min_w = 0
    chart_max_w = max_w_data + w_margin * 2.0
    w_sat_at_max_temp_chart = get_humidity_ratio_kg_kg(chart_max_temp, 100)
    if w_sat_at_max_temp_chart != float('inf'):
        chart_max_w = max(chart_max_w, w_sat_at_max_temp_chart * 1.05)
    chart_max_w = min(chart_max_w, 0.035)

    fig = go.Figure()
    add_psychrometric_background_to_fig(fig, (chart_min_temp, chart_max_temp), (chart_min_w, chart_max_w))

    fig.add_trace(go.Scatter(
        x=[], y=[], mode='markers', marker=dict(size=10), name="공조 과정 지점",
        text=[], hovertemplate="<b>%{text}</b><br>온도: %{x:.1f}°C<br>절대습도: %{y:.4f} kg/kg<br>상대습도: %{customdata[0]:.0f}%<br>엔탈피: %{customdata[1]:.1f} kJ/kg<extra></extra>",
        customdata=[], showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=[], y=[], mode='lines', line=dict(color='gray', width=1.5, dash='solid'), name="공조 과정 경로",
        hoverinfo='skip', showlegend=False
    ))

    frames = []
    for idx, row_data in df.iterrows():
        timestamp_str = row_data['Timestamp'].strftime('%Y-%m-%d %H:%M:%S')
        frame_points_x = []
        frame_points_y = []
        frame_points_text = []
        frame_points_customdata = []
        frame_points_colors = []
        current_annotations = []

        for i, stage_conf in enumerate(mapped_stages_config):
            try:
                temp_c = row_data[stage_conf['temp_col']]
                rh_p = row_data[stage_conf['rh_col']]
            except KeyError:
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_points_text.append(stage_conf['name'] + " (데이터 오류)")
                frame_points_customdata.append([None, None])
                frame_points_colors.append('grey')
                continue

            w = get_humidity_ratio_kg_kg(temp_c, rh_p)
            h = get_enthalpy_kJ_kg(temp_c, rh_p)

            if w != float('inf') and h != float('inf') and w < chart_max_w * 1.5 :
                frame_points_x.append(temp_c)
                frame_points_y.append(w)
                frame_points_text.append(stage_conf['name'])
                frame_points_customdata.append([rh_p, h])
                frame_points_colors.append(stage_conf['color'])
                current_annotations.append(dict(
                    x=temp_c, y=w,
                    text=f"<b>{stage_conf['name']}</b><br>{temp_c}°C, {rh_p}%<br>h={h:.1f}",
                    showarrow=True, arrowhead=1, arrowwidth=1, arrowcolor=stage_conf['color'],
                    ax=20, ay=-30, font=dict(size=9, color='black'),
                    bordercolor=stage_conf['color'], borderwidth=1, bgcolor="rgba(255,255,255,0.8)" ))
            else:
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_points_text.append(stage_conf['name'] + " (계산 오류)")
                frame_points_customdata.append([None, None])
                frame_points_colors.append('grey')

        frames.append(go.Frame(
            name=timestamp_str,
            data=[
                go.Scatter(x=frame_points_x, y=frame_points_y, text=frame_points_text,
                           customdata=np.array(frame_points_customdata).T if frame_points_customdata and any(cd is not None for cd_list in frame_points_customdata for cd in cd_list) else [],
                           marker=dict(color=frame_points_colors, size=10, line=dict(width=1, color='DarkSlateGrey'))),
                go.Scatter(x=frame_points_x, y=frame_points_y, line=dict(color='dimgray'))
            ],
            layout=go.Layout(
                annotations=current_annotations,
                title_text=f"대화형 습공기선도: {selected_filename} - {timestamp_str}" )))

    fig.frames = frames
    sliders = [dict(active=0, currentvalue={"prefix": "시간: ", "font": {"size": 14}}, pad={"t": 50, "b":10}, steps=[])]
    for i, timestamp in enumerate(df['Timestamp']):
        ts_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        slider_step = dict(label=timestamp.strftime('%H:%M'), method="animate",
                           args=[[ts_str], dict(mode="immediate", frame=dict(duration=100, redraw=True),
                                                transition=dict(duration=50))])
        sliders[0]['steps'].append(slider_step)

    initial_timestamp_str = df['Timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M:%S')
    fig.update_layout(
        title=f"대화형 습공기선도: {selected_filename} - {initial_timestamp_str}",
        xaxis=dict(title='건구온도 (°C)', range=[chart_min_temp, chart_max_temp]),
        yaxis=dict(title='절대습도 (kg_water / kg_dry_air)', range=[chart_min_w, chart_max_w]),
        sliders=sliders, legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, traceorder="normal"),
        margin=dict(l=60, r=60, b=120, t=100), hovermode="closest" )

    if frames:
        first_frame = frames[0]
        points_trace_index = len(fig.data) - 2 # 공조 과정 지점 트레이스
        lines_trace_index = len(fig.data) - 1  # 공조 과정 경로 트레이스

        fig.data[points_trace_index].x = first_frame.data[0].x
        fig.data[points_trace_index].y = first_frame.data[0].y
        fig.data[points_trace_index].text = first_frame.data[0].text
        fig.data[points_trace_index].customdata = first_frame.data[0].customdata
        if hasattr(first_frame.data[0], 'marker') and hasattr(first_frame.data[0].marker, 'color'):
             fig.data[points_trace_index].marker.color = first_frame.data[0].marker.color
        fig.data[lines_trace_index].x = first_frame.data[1].x
        fig.data[lines_trace_index].y = first_frame.data[1].y
        if hasattr(first_frame.layout, 'annotations'):
             fig.layout.annotations = first_frame.layout.annotations
        else:
             fig.layout.annotations = []
    fig.show()

# --- 더미 CSV 데이터 (컬럼명 다양화) ---
dummy_csv_data_ahu1_oac = """Timestamp,AHU01_PH/C_온도_입구,AHU01_PH/C_습도_입구,AHU01_PC/C_Zone1_온도,AHU01_PC/C_Zone1_습도,AHU01_CoolingCoil_Air_온도_Avg,AHU01_CoolingCoil_Air_습도_Avg,AHU01_HC_승온후_온도값,AHU01_HC_승온후_습도값
2023-07-15 10:00:00,30.0,65,27.0,75,16.0,92,21.0,68
2023-07-15 10:10:00,30.2,66,26.5,76,14.0,96,23.0,58
2023-07-15 10:20:00,30.5,64,26.0,77,12.0,99,25.0,48
2023-07-15 11:00:00,31.0,65,26.0,77,12.5,98,25.5,47
2023-07-15 11:10:00,31.2,66,25.5,78,13.0,97,26.0,46
"""
dummy_csv_data_ahu2_oac_control = """Timestamp,ZoneA_PREHEAT_온도,ZoneA_PREHEAT_습도,ZoneA_PRECOOL_온도,ZoneA_PRECOOL_습도,MainUnit_C/C_온도_출구,MainUnit_C/C_습도_출구,PostZone_HEAT_온도,PostZone_HEAT_습도
2023-07-16 14:00:00,28.0,70,25.0,80,17.0,88,20.0,65
2023-07-16 14:10:00,28.2,72,24.5,81,16.0,93,22.5,55
2023-07-16 14:20:00,28.5,71,24.0,82,15.0,97,24.5,45
2023-07-16 15:00:00,29.0,70,24.0,82,15.5,96,25.0,44
"""
dummy_csv_data_no_oac = """Timestamp,Sensor_Temp,Sensor_RH
2023-07-17 09:00:00,22,55
"""
VIRTUAL_FILE_SYSTEM = {
    "data_ahu1_oac.csv": dummy_csv_data_ahu1_oac,
    "data_ahu2_oac_control.csv": dummy_csv_data_ahu2_oac_control,
    "other_sensor_data.csv": dummy_csv_data_no_oac,
    "report_oac_summary.txt": "이것은 CSV가 아닙니다."
}

# --- 실행 부분 ---
if __name__ == '__main__':
    available_oac_files = find_oac_csv_files_simulated(VIRTUAL_FILE_SYSTEM)
    selected_filename, selected_csv_content = select_csv_file_simulated(available_oac_files)

    if selected_csv_content:
        print(f"\n선택된 시뮬레이션 파일에서 데이터 로딩 중: {selected_filename}")
        try:
            df_loaded = pd.read_csv(io.StringIO(selected_csv_content))
            if 'Timestamp' not in df_loaded.columns:
                raise ValueError("CSV 파일에 'Timestamp' 컬럼이 반드시 포함되어야 합니다.")
            df_loaded['Timestamp'] = pd.to_datetime(df_loaded['Timestamp'])

            print(f"원본 데이터 포인트 수: {len(df_loaded)}")
            df_loaded.set_index('Timestamp', inplace=True)

            # 숫자형 컬럼만 선택하여 리샘플링 (평균값 사용)
            numeric_cols = df_loaded.select_dtypes(include=np.number).columns
            if not numeric_cols.empty:
                 df_resampled = df_loaded[numeric_cols].resample('1H').mean(numeric_only=True)
            else: # 숫자형 컬럼이 없는 경우, 원본 사용 (또는 다른 처리)
                 print("경고: 데이터에 숫자형 컬럼이 없어 리샘플링을 건너<0xEB><0><0x8A>니다.")
                 df_resampled = df_loaded.copy() # 원본 데이터프레임 복사

            df_resampled.reset_index(inplace=True) # Timestamp를 다시 컬럼으로
            print(f"1시간 단위 리샘플링 후 데이터 포인트 수: {len(df_resampled)}")

            mapped_stages = map_columns_from_csv(df_resampled.columns)

            if mapped_stages:
                create_interactive_psychrometric_chart_plotly(df_resampled, mapped_stages, selected_filename)
            else:
                print("\n컬럼 매핑 문제로 차트 생성을 진행할 수 없습니다.")

        except Exception as e:
            print(f"파일 {selected_filename} 처리 중 오류 발생: {e}")
    else:
        print("파일이 선택되지 않았거나 오류가 발생했습니다. 프로그램을 종료합니다.")

    # --- 실제 파일 시스템 사용 예시 (주석 처리됨) ---
    # DATA_DIRECTORY = "./my_csv_data/" # 데이터 디렉토리 정의
    # actual_oac_files = []
    # if os.path.exists(DATA_DIRECTORY):
    #     for filename in os.listdir(DATA_DIRECTORY):
    #         if "oac" in filename.lower() and filename.endswith(".csv"):
    #             actual_oac_files.append(os.path.join(DATA_DIRECTORY, filename))
    #
    # if not actual_oac_files:
    #     print(f"디렉토리에서 'OAC' CSV 파일을 찾을 수 없습니다: {DATA_DIRECTORY}")
    # else:
    #     print("\n사용 가능한 'OAC' CSV 파일:")
    #     for i, filepath in enumerate(actual_oac_files):
    #         print(f"  {i+1}. {os.path.basename(filepath)}")
    #     # ... (이하 파일 선택 및 처리 로직 동일하게 적용) ...

시뮬레이션된 파일 검색 중...

사용 가능한 'OAC' CSV 파일 (시뮬레이션):
  1. data_ahu1_oac.csv
  2. data_ahu2_oac_control.csv
파일 번호를 선택하세요 (1-2): 1

선택된 시뮬레이션 파일에서 데이터 로딩 중: data_ahu1_oac.csv
원본 데이터 포인트 수: 5
1시간 단위 리샘플링 후 데이터 포인트 수: 2

컬럼 매핑 시도 중...
  스테이지 매핑 성공: 1. PH/C (예열)
    온도 컬럼: 'AHU01_PH/C_온도_입구'
    습도 컬럼: 'AHU01_PH/C_습도_입구'
  스테이지 매핑 성공: 2. PC/C (예냉)
    온도 컬럼: 'AHU01_PC/C_Zone1_온도'
    습도 컬럼: 'AHU01_PC/C_Zone1_습도'
  스테이지 매핑 성공: 3. C/C (냉각)
    온도 컬럼: 'AHU01_CoolingCoil_Air_온도_Avg'
    습도 컬럼: 'AHU01_CoolingCoil_Air_습도_Avg'
  스테이지 매핑 성공: 4. H/C (가열/재열)
    온도 컬럼: 'AHU01_HC_승온후_온도값'
    습도 컬럼: 'AHU01_HC_승온후_습도값'


<ipython-input-1-95632f79b82b>:384: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df_loaded[numeric_cols].resample('1H').mean(numeric_only=True)
